In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [0]:
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz

In [0]:
!tar xf spark-2.4.0-bin-hadoop2.7.tgz

In [0]:
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('blackwaterbattle').getOrCreate()

In [0]:
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.regression import RandomForestRegressor

In [21]:
cd ../content/sample_data/b

/content/sample_data


In [0]:
input_data = spark.read.csv('/content/sample_data/train.csv', header=True, inferSchema=True)

In [0]:
input_test_data = spark.read.csv('/content/sample_data/test.csv', header=True, inferSchema=True)

In [0]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.ml.evaluation import RegressionEvaluator

In [72]:
input_data.count()

1941261

In [0]:
assembler = VectorAssembler(inputCols=['shipId', 'attackId', 'assists', 'greekFireItems', 'healthLost', 'knockedOutSoldiers', 'throatSlits', 'healingPotionsUsed', 'killRank', 'killPoints', 'enemiesKilled', 'killingStreaks', 'farthermostKill', 'numShips', 'numSaves', 'horseRideDistance', 'horseRideKills', 'swimmingDistance', 'friendlyKills', 'castleTowerDestroys', 'onFootDistance', 'weaponsUsed', 'respectEarned'], outputCol='features')

In [0]:
output_data = assembler.transform(input_data)
final_data = output_data.select('features', 'bestSoldierPerc')

In [0]:
train,test = final_data.randomSplit([0.7,0.3])

In [63]:
model = GBTRegressor(labelCol='bestSoldierPerc',lossType='absolute', maxIter=50, stepSize=0.05, seed=None, featureSubsetStrategy='auto')
modelx=model.fit(train)
predictions = modelx.transform(cv)
evaluator = RegressionEvaluator(labelCol="bestSoldierPerc", predictionCol="prediction", metricName="mae")
mae = evaluator.evaluate(predictions)
print(mae)

0.07312995767794105


In [60]:
#modelr = RandomForestRegressor(labelCol='bestSoldierPerc',numTrees=4, featureSubsetStrategy="all",impurity='variance', maxDepth=9, maxBins=32)
#modely=modelr.fit(train)
predictions = modely.transform(cv)
evaluator = RegressionEvaluator(labelCol="bestSoldierPerc", predictionCol="prediction", metricName="mae")
mae = evaluator.evaluate(predictions)
print(mae)

0.08360972264015169


In [0]:
output_test_data = assembler.transform(input_test_data)
final_test_data = output_test_data.select('features')

In [0]:
model = GBTRegressor(labelCol='bestSoldierPerc',lossType='absolute', maxIter=50, stepSize=0.05, seed=None, featureSubsetStrategy='auto')
modelx=model.fit(final_data)
predictions = modelx.transform(final_test_data)

In [0]:
res = predictions.select('prediction')

In [0]:
res.write.csv('out.csv')